In [1]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer
from transformers import AutoConfig
from transformers import AutoModelForSequenceClassification, BertForSequenceClassification

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [3]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

model = torch.load('weights/bert_sentiment_analysis.pth')

model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [4]:
def tokenize_text(text):
    inputs = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=100,
        padding='max_length',
        truncation=True
    )

    return {
        'ids': torch.tensor(inputs['input_ids'], dtype=torch.long).unsqueeze_(0),
        'mask': torch.tensor(inputs['attention_mask'], dtype=torch.long).unsqueeze_(0),
        'token_type_ids': torch.tensor(inputs["token_type_ids"], dtype=torch.long).unsqueeze_(0)
    }

In [9]:
sentiment_text = input('Type Here: ')

tokenized_text = tokenize_text(sentiment_text)

with torch.no_grad():
    text = tokenized_text['ids'].to(device)
    token_ids = tokenized_text['token_type_ids'].to(device)
    mask = tokenized_text['mask'].to(device)
    output = model(
        text,
        token_type_ids=token_ids,
        attention_mask=mask
    )[0]
    
    text = text.to('cpu')
    token_ids = token_ids.to('cpu')
    mask = mask.to('cpu')

output = F.softmax(output.detach().cpu(), dim=-1).flatten().numpy() * 100

print('Sentiment Text: {}'.format(sentiment_text))
print('Negative: {} \nPositive: {}%'.format(round(output[0], 2), round(output[1], 2)))

Type Here: life is emotionless and life is hard
Sentiment Text: life is emotionless and life is hard
Negative: 85.30999755859375 
Positive: 14.6899995803833%
